In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')
df

In [ ]:
X = df.drop(labels = ['RainToday', 'RainTomorrow'], axis = 1)
Y = df['RainToday']
print(X.shape, Y.shape)

In [ ]:
X.drop(labels = 'Date',axis = 1, inplace =True)

In [ ]:
X

In [ ]:
X.info()

In [ ]:
numeric = X._get_numeric_data()

In [ ]:
from sklearn.feature_selection import VarianceThreshold
vr = VarianceThreshold(threshold=0)
vr.fit(numeric)
vr.get_support()

In [ ]:
numeric.isnull().sum()

In [ ]:
numeric.hist(figsize=(20, 20))

In [ ]:
numeric.describe()

In [ ]:
test = pd.DataFrame(numeric['MinTemp'])

arr = pd.DataFrame(np.where(test['MinTemp'] > 7, 7, test['MinTemp']), columns=['MinTemp'])
li_q1 = []
li_q3 = []
def find_q1_q3(numeric):
    for i in numeric:
        q1 = np.percentile(X[i], 25)
        q3 = np.percentile(X[i], 75)
        li_q1.append(q1)
        li_q3.append(q3)
    

for i in numeric:
    numeric[i] = numeric[[i]].replace(np.nan, numeric[i].median())
find_q1_q3(numeric)
# arr = pd.DataFrame(np.where(test['MinTemp'] == 13.4, 10, test['MinTemp']), columns=['MinTemp'])

for j,i in enumerate(numeric):
    numeric[i] = pd.DataFrame(np.where(numeric[i] > li_q3[j] + 1.5 * (li_q3[j] - li_q1[j]), li_q3[j], numeric[i]), columns=[i])

numeric.hist(figsize=(20,20))

In [ ]:
numeric.describe()

In [ ]:
for j,i in enumerate(numeric):
    numeric[i] = pd.DataFrame(np.where(numeric[i] < li_q1[j] - 1.5 * (li_q3[j] - li_q1[j]), li_q1[j], numeric[i]), columns=[i])


In [ ]:
numeric.hist(figsize=(20,20))

In [ ]:
X

In [ ]:
l_1 = list(numeric.columns.values)

res = list(map(lambda x: x not in l_1, X.columns.values ))
res

In [ ]:
string_data = X.loc[:, res]

In [ ]:
string_data.info()

In [ ]:
string_data[['WindGustDir']].replace(np.nan, string_data['WindGustDir'].mode())
string_data['WindDir3pm'] = string_data[['WindDir3pm']].replace(np.nan, string_data['WindDir3pm'].mode(), inplace=True)
string_data['WindDir9am'] = string_data[['WindDir9am']].replace(np.nan, string_data['WindDir9am'].mode(), inplace=True)

In [ ]:
string_data

In [ ]:
string_data.isnull().sum()

In [ ]:
string_data.loc[:, 'WindgustDir'] = string_data['WindGustDir'].replace(np.nan, 'W')
string_data.loc[:, 'WindDir3pm'] = string_data['WindDir3pm'].replace(np.nan, 'SE')
string_data.loc[:, 'WindDir9am'] = string_data['WindDir9am'].replace(np.nan, 'N')

In [ ]:
corr_matrix = numeric.corr()
plt.figure(figsize=(8,6))
plt.title('Correlation Heatmap of rain aus Dataset')
sns.heatmap(corr_matrix, annot=True, square=True, linecolor='black')
plt.show()

In [ ]:
p = pd.DataFrame(np.triu(corr_matrix,k = 1))
p

In [ ]:
high_corr_col = [col for col in p.columns if any(p[col] > 0.9) ]


In [ ]:
numeric.drop(labels=numeric.iloc[:,[11, 15]].columns.values, axis = 1, inplace = True)
numeric

In [ ]:
string_data = pd.get_dummies(string_data, drop_first=True)

In [ ]:
X = pd.concat([numeric, string_data], axis = 1)

In [ ]:
X

In [ ]:
d = pd.concat([X, Y], axis = 1)
d.dropna(inplace=True)
X = d.drop('RainToday', axis = 1)
Y = d[['RainToday']]
# Y.replace(to_replace =['Yes'])

In [ ]:
Y.value_counts()
Y.replace(to_replace=['Yes', 'No'], value = [1, 0], inplace=True)


In [ ]:
X.dtypes.value_counts()

In [ ]:
Y.value_counts()
Y.astype('int')

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X = scale.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X, x, Y, y = train_test_split(X, Y, test_size=0.2, random_state=3)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
li = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
solvers_accuracy = {}
lr = None
for v in li:
    lr = LogisticRegression(C = 0.01, solver=v)
    lr = lr.fit(X, Y.values.ravel())
    pred = lr.predict(x)
    
    solvers_accuracy.update({v:round(metrics.accuracy_score(y, pred), 2)})

solvers_accuracy

In [ ]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(C = 0.01, solver='liblinear')
lg.fit(X, Y.values.ravel())
y_pred = lg.predict(x)


In [ ]:
conf_metr = metrics.confusion_matrix(y, lr.predict(x))
conf_metr

In [ ]:
print("Recall: {}".format((conf_metr[0,0]/(conf_metr[0,0]+conf_metr[1,0]))))

print("Precision: {}".format((conf_metr[0,0]/(conf_metr[0,0]+conf_metr[0,1]))))

print("True nPositive Rate: {}".format((conf_metr[0,0]/(conf_metr[0,0]+conf_metr[1,0]))))

print("Fale Positive Rate: {}".format((conf_metr[0,1]/(conf_metr[0,1]+conf_metr[1,1]))))

In [ ]:
metrics.plot_roc_curve(lg, X, Y)

In [ ]:
metrics.roc_auc_score(y, lg.predict(x))